# Metody Inteligencji Obliczeniowej w Analizie Danych

[Tasks](https://pages.mini.pw.edu.pl/~karwowskij/mioad/lab-sieci.html)

### Lab 2/3:  Implementacja propagacji wstecznej błędu (2 tygodnie, 4 punkty)

W ramach tego laboratorium trzeba zaimplementować uczenie sieci neuronowej propagacją wsteczną błędu.


Aby sprawdzić implementację, należy wykonać uczenie na prostych danych do uczenia dostarczonych na
zajęciach. Następnie należy zaimplementować metodę wizualizacji wartości wag sieci w kolejnych iteracjach
i, w przypadku gdy nie udaje się nauczyć sieci, spróbować wykorzystać te wizualizacje do ustalenia
przyczyny problemu. Zaimplementować wersję z aktualizacją wag po prezentacji wszystkich wzorców i
wersję z aktualizacją po prezentacji kolejnych porcji (batch). Porównać szybkość uczenia dla każdego z
wariantów.

Inicjować wagi z rozkładu jednostajnego na przedziale [0,1]. Opcjonalnie zaimplementować inną metodą
inicjowania wag. Albo metodę He albo Xavier.

Przetestować uczenie sieci na następujących zbiorach (w nawiasach wartość MSE, aby otrzymać 4 punkty):
- square-simple (jeśli sieć nie jest w stanie się nauczyć tej funkcji to coś jest bardzo źle zrobione, MSE:
4),
- steps-small (MSE: 4),
- multimodal-large (MSE: 40).
(Wartości MSE liczone na nieznormalizowanych danych)

#### TODO:

- ~~change the weights initialization to uniform(0,1), Xavier or He~~
- implemnent cost function for MSE
- ~~implement gradients for activation functions~~
- implement backpropagation
- implement batch learning
- implement visualization
- implement visualization for batch learning
- compare learning speed with and without batch learning


In [1]:
from MultiLayerPerceptron import NeuralNetwork, Layer
from itertools import chain
import pandas as pd
import numpy as np

def mse(real, pred):
    return np.square(np.subtract(real,pred)).mean() 

In [3]:
np.random.seed(1)
# get matrix 3x5 from random uniform
np.random.uniform(size=(3,5))

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01],
       [9.23385948e-02, 1.86260211e-01, 3.45560727e-01, 3.96767474e-01,
        5.38816734e-01],
       [4.19194514e-01, 6.85219500e-01, 2.04452250e-01, 8.78117436e-01,
        2.73875932e-02]])

## Dataset 2: Square Simple

In [2]:
df_train = pd.read_csv('data/regression/square-simple-training.csv').set_index("Unnamed: 0")
df_test = pd.read_csv('data/regression/square-simple-test.csv').set_index("Unnamed: 0")
print(df_train.head())

x_train = [[x] for x in df_train.loc[:,"x"]]
y_train = [y for y in df_train.loc[:,"y"]]
x_test = [[x] for x in df_test.loc[:,"x"]]
y_test = [y for y in df_test.loc[:,"y"]]

                   x           y
Unnamed: 0                      
1          -0.171543 -127.351580
2           0.025201 -129.942844
3          -1.368991   38.672367
4           1.907390  197.432191
5           0.011129 -129.988852


#### Model 1: 1 hidden layer with 5 neurons

In [3]:
net1_1_5_1 = NeuralNetwork(weights_random=False)
net1_1_5_1.add(Layer(neurons_count=1, add_bias=True))
net1_1_5_1.add(Layer(neurons_count=5, activation_fun="sigmoid", add_bias=True))
net1_1_5_1.add(Layer(neurons_count=1, activation_fun="linear", add_bias=False))
net1_1_5_1.set_weights(weights=[\
        np.array([[-2.9,-2.8897,-2.8897,-2.8897,-2.8897],\
                [-0.9733,-0.9733,-0.9733,-0.9733,0.9733]]),\
        np.array([[-339.16],\
                [502],\
                [502],\
                [502],\
                [502],\
                [1993]])])
print(net1_1_5_1)

Neural network layers:
	Layer 1: Layer has 2 neurons (including 1 bias neuron) and activation function is 'linear function'
	Layer 2: Layer has 6 neurons (including 1 bias neuron) and activation function is 'sigmoid function'
	Layer 3: Layer has 1 neurons (with no bias neuron) and activation function is 'linear function'
Neural network weights:
	Weights 1: (2, 5) (input, output)
[[-2.9    -2.8897 -2.8897 -2.8897 -2.8897]
 [-0.9733 -0.9733 -0.9733 -0.9733  0.9733]]
	Weights 2: (6, 1) (input, output)
[[-339.16]
 [ 502.  ]
 [ 502.  ]
 [ 502.  ]
 [ 502.  ]
 [1993.  ]]



In [4]:
y_train_pred_1 = net1_1_5_1.predict(x_train)
y_train_pred_1 = list(chain.from_iterable(y_train_pred_1))
mse_1_1 = mse(y_train_pred_1, y_train)
print(f"MSE train is {round(mse_1_1,2)}")

MSE train is 2.46


In [5]:
y_test_pred_1 = net1_1_5_1.predict(x_test)
y_test_pred_1 = list(chain.from_iterable(y_test_pred_1))
mse_1_2 = mse(y_test_pred_1, y_test)
print(f"MSE test is {round(mse_1_2,2)}")

MSE test is 2.26


## Dataset 2: Steps Small

In [9]:
df_train = pd.read_csv('data/regression/steps-large-training.csv').set_index("Unnamed: 0")
df_test = pd.read_csv('data/regression/steps-large-test.csv').set_index("Unnamed: 0")
print(df_train.head())

x_train = [[x] for x in df_train.loc[:,"x"]]
y_train = [y for y in df_train.loc[:,"y"]]
x_test = [[x] for x in df_test.loc[:,"x"]]
y_test = [y for y in df_test.loc[:,"y"]]

                   x   y
Unnamed: 0              
1          -1.481354 -80
2           1.033264  80
3          -0.076403   0
4          -1.419785 -80
5          -0.108398   0


#### Model 1: 2 hidden layers with 5 neurons each

In [10]:
net3_1_5_1 = NeuralNetwork()
net3_1_5_1.add(Layer(neurons_count=1, add_bias=True))
net3_1_5_1.add(Layer(neurons_count=5, activation_fun='sigmoid', add_bias=True))
net3_1_5_1.add(Layer(neurons_count=1, activation_fun='linear', add_bias=False))

b = -250
a = 500
c = -80
d = 80
e = 80/2

net3_1_5_1.set_weights([\
        np.array([[-250,-750,-750,250,250],\
                [501,499.95,499.95,500.2,500.2]]),\
        np.array([[-80],\
                [80],\
                [40],\
                [40],\
                [40],\
                [40]])])
print(net3_1_5_1)

Neural network layers:
	Layer 1: Layer has 2 neurons (including 1 bias neuron) and activation function is 'linear function'
	Layer 2: Layer has 6 neurons (including 1 bias neuron) and activation function is 'sigmoid function'
	Layer 3: Layer has 1 neurons (with no bias neuron) and activation function is 'linear function'
Neural network weights:
	Weights 1: (2, 5) (input, output)
[[-250.   -750.   -750.    250.    250.  ]
 [ 501.    499.95  499.95  500.2   500.2 ]]
	Weights 2: (6, 1) (input, output)
[[-80]
 [ 80]
 [ 40]
 [ 40]
 [ 40]
 [ 40]]



In [11]:
y_train_pred_2 = net3_1_5_1.predict(x_train)
y_train_pred_2 = list(chain.from_iterable(y_train_pred_2))
mse_1_2 = mse(y_train_pred_2, y_train)
print(f"MSE train is {round(mse_1_2,2)}")

MSE train is 4.14


In [12]:
y_test_pred_2 = net3_1_5_1.predict(x_test)
y_test_pred_2 = list(chain.from_iterable(y_test_pred_2))
mse_1_2 = mse(y_test_pred_2, y_test)
print(f"MSE test is {round(mse_1_2,2)}")

MSE test is 3.58


## Multimodal Large

# TODO

#### Results

MSE for the `square simple` dataset is:  
- train: 2.46
- test: 2.26

MSE for the `steps large` dataset is:  
- train: 4.14
- test: 3.58


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=96d48dca-8d08-48ed-b693-a040059620ca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>